In [2]:
#### Let's start our notebook with typical and Important Imports
import os
import _pickle as cPickle
import numpy as np
from scipy.io.wavfile import read
from sklearn.mixture import GaussianMixture
import python_speech_features as pySpeech 
### You might have to $ pip install python_speech_features
from sklearn import preprocessing
import warnings
warnings.filterwarnings("ignore")

In [3]:
## Define MFCC fuction : The Mel Frequency Cepstral Coefficient
## Plz refer :: http://practicalcryptography.com/miscellaneous/machine-learning/guide-mel-frequency-cepstral-coefficients-mfccs/
def get_MFCC(sr,audio):
    features = pySpeech.mfcc(audio,sr, 0.025, 0.01, 13,appendEnergy = False)
    features = preprocessing.scale(features)
    return features

In [6]:
### Here load the already separated Male and female voice 
### this serve as analogous to labelled data 
with open("maleaudios") as rf:
    filesM = [i.strip() for i in rf.readlines()]

with open("femaleaudios") as rf:
    filesF = [i.strip() for i in rf.readlines()]

In [4]:
featuresM = np.asarray(())
featuresF = np.asarray(())

In [7]:
#Female Voice specific Feature engineering 
for f in filesF[:10000]:
    try:
        sr,audio  = read(m)
        vectorF   = get_MFCC(sr,audio)
        if featuresF.size == 0:
            featuresF = vectorF
        else:
            featuresF = np.vstack((featuresF, vectorF))
    except Exception as e:
        print("Exception in file {}".format(f))

In [8]:
#Male Voice specific Feature engineering 
for m in filesM[:10000]:
    try:
        sr,audio  = read(m)
        vectorM   = get_MFCC(sr,audio)
        if featuresM.size == 0:
            featuresM = vectorM
        else:
            featuresM = np.vstack((featuresM, vectorM))
    except Exception as e:
        print("Exception in file {}".format(m))


In [9]:
#GMMs models :: Separate Models for male and Female
female_model = GaussianMixture(n_components = 11, max_iter = 350, covariance_type='diag', n_init = 5)
male_model = GaussianMixture(n_components = 11, max_iter = 350, covariance_type='diag', n_init = 5)
female_model.fit(featuresF)
male_model.fit(featuresM)


In [10]:
# Training is done by now :: Jar up the Pickle
cPickle.dump(female_model,open("female.gmm",'wb'))
cPickle.dump(male_model,open("male.gmm",'wb'))